# Importing all the necessary libraires

In [2]:
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np 
import requests 
import re
import pandas as pd
import nltk
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Defining header

In [3]:
headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}

# Getting the URL and iterating through the 10 pages to extract links

In [4]:
URL = 'https://www.brookings.edu/topic/terrorism-extremism/page/'
lis=[]
for page in range(1,10):
    
    req = requests.get(URL + str(page) + '/?type=posts')
    
    soup = BeautifulSoup(req.text,'lxml')
    links = [link['href'] for link in soup.find_all('a', href=re.compile('^(https://www.brookings.edu/blog/order-from-chaos/)'))]
   
    links=list(set(links))
    
    lis.append(links)
     
    


In [5]:
len(lis)


9

# Inserting all scraped links in a single list

In [6]:
list0=lis[0]
list1=lis[1]
list2=lis[2]
list3=lis[3]
list4=lis[4]
list5=lis[5]
list6=lis[6]
list7=lis[7]
list8=lis[8]

finalList=list0+list1+list2+list3+list4+list5+list6+list7+list8


#finalList
len(finalList)


    

76

In [7]:
finalList

['https://www.brookings.edu/blog/order-from-chaos/2021/09/20/i-hadnt-the-slightest-idea-my-life-would-change-after-9-11/',
 'https://www.brookings.edu/blog/order-from-chaos/2021/09/07/reflections-on-the-long-term-repercussions-of-september-11-for-us-policy-in-the-middle-east/',
 'https://www.brookings.edu/blog/order-from-chaos/2021/09/08/the-tel-aviv-plot/',
 'https://www.brookings.edu/blog/order-from-chaos/2021/09/08/the-impact-of-september-11-on-us-russian-relations/',
 'https://www.brookings.edu/blog/order-from-chaos/2021/09/17/9-11-and-iraq-the-making-of-a-tragedy/',
 'https://www.brookings.edu/blog/order-from-chaos/2021/09/28/the-agonizing-problem-of-pakistans-nukes/',
 'https://www.brookings.edu/blog/order-from-chaos/2021/10/01/what-anwar-sadats-murder-40-years-ago-meant-for-the-middle-east/',
 'https://www.brookings.edu/blog/order-from-chaos/2021/11/19/is-hezbollah-overplaying-its-hand-inside-lebanon/',
 'https://www.brookings.edu/blog/order-from-chaos/2021/08/31/will-the-taliba

# Iterating through the links in finalList and extracting text

In [8]:
x=[]
for url in finalList:
    # getting the reponse from the page using get method of requests module
    page = requests.get(url, headers=headers)

    # storing the content of the page in a variable
    html = page.content

    # creating BeautifulSoup object
    soup = BeautifulSoup(html, "html.parser")
    
    text=[p.text for p in soup.find(class_="post-body post-body-enhanced").find_all('p')]
    x.append(text)
    arr=np.array(x ,dtype="object")
    

In [9]:
df = pd.DataFrame(arr, columns=['raw_data'])



In [10]:
df

,raw_data
0,[I hadn’t the slightest idea that my life woul...
1,[It is one of the ironies often played by hist...
2,[Recently declassified information from the fi...
3,"[On September 9, 2001, Russian President Vladi..."
4,[Twenty years after the al-Qaida attack on Sep...
...,...
71,"[On August 3, a shooter opened fire at a crowd..."
72,[The Islamic State has claimed responsibility ...
73,"[The release of John Walker Lindh, the “Americ..."
74,[As dozens of countries develop national frame...


In [11]:
df.to_csv("scrapped-raw-articles.csv", index=False, encoding='utf-8')

# Data Cleaning


In [12]:
print(x[2])

['Recently declassified information from the first-ever interrogation of someone presumed to be a senior al-Qaida operative captured after 9/11 provides dramatic new insights into Osama bin Laden’s plans for a follow-up attack to September 11. Specifically, bin Laden was plotting a major attack in Israel, a move consistent with his obsession with the Arab-Israeli conflict and U.S. support for Israel. The attack was thwarted at the last minute. So why is this coming to light only 20 years later?\t\n\n\n\n\n\n\nBruce Riedel\n\n\t\t\t\t\tSenior Fellow - Foreign Policy, Center for Middle East Policy, Center for Security, Strategy, and Technology \n\n\t\t\t\t\tDirector - The Intelligence Project \n\n\n\n\n', 'Zayn al-Abidin Muhammad Husayn, also known as Abu Zubaydah, was captured in Pakistan in March 2002. Abu Zubaydah is a Palestinian born in Saudi Arabia who was allegedly involved in preparing the travel and documentation of many al-Qaida operatives. Not technically a member of the group

In [13]:
len(x)


76

In [14]:
documents=[]
stemmer =WordNetLemmatizer()
for sen in range(0, len(x)):
    #Remove all special characters
    document = re.sub(r'\w',' ',str(x[sen]))
    
    document = re.sub("\\\\.|[^a-zA-Z' ]+", '',str(x[sen]))
    
    #remove all punctuations
    document = re.sub(re.escape(string.punctuation), '' , document)
    
    #remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+','',document)
    
    #remove all single characters from start
    document = re.sub(r'\^[a-zA-Z]\s+',' ',document)
    
    #substituting multiple spaces with single space
    document = re.sub(r'\s+'," ",document, flags=re.I)
    
    #removing prefixed b
    document = re.sub(r'^b\s+','',document)
    
    document=re.sub(r'\'',"",document)
    document = document.lower()
    
    doucment = document.split()
    
    document =[stemmer.lemmatize(word) for word in document]
    document = ''.join(document)
    doucment = [word for word in document.split() if word not in stopwords.words("english")]
    document = ''.join(document)
    document=np.array(document)
    documents.append(document)
   
    
    

In [15]:
documents[0]
#type(documents)    
    

array('i hadnt the slightest idea that my life would change it was two weeks into my freshman year of collegewas living away from home for the first time and then the september attacks happened it felt likedouble tragedy to meas an american of course but also asmuslim not only did they do this they did this in my nametshadi hamidsenior fellow foreign policy center for middle east policy twittershadihamid before osama bin laden becamehousehold namequaint thought in retrospect it wouldnt have occurred to me that my faith would soon become inextricably linked to terrorism in the publics imagination in high school there was that one time whenkid named dan called meterrorist butdont recall it beingbig deal no one really cared that we were muslim sure we were brownwas different in an overwhelmingly white school but we were morecuriosity and not quitethreat in retrospect im impressed that dan was sufficiently aware of global affairs to have already perceived an association between islam and t

In [17]:
df=pd.DataFrame()
df['orignal'] = x
df["Cleaned"] = documents
df.to_csv("clean_data.csv", index = False)
df=pd.read_csv('clean_data.csv')
df.head()

,orignal,Cleaned
0,['I hadn’t the slightest idea that my life wou...,i hadnt the slightest idea that my life would ...
1,['It is one of the ironies often played by his...,it is one of the ironies often played by histo...
2,['Recently declassified information from the f...,recently declassified information from the fir...
3,"['On September 9, 2001, Russian President Vlad...",on september russian president vladimir putin ...
4,['Twenty years after the al-Qaida attack on Se...,twenty years after the alqaida attack on septe...


In [18]:
len(df)

76